# Name: 
Faith Jeptoo
# Title: 
Data Wrangling Project
# Date: 
5 October, 2025
# CyberShujaa ID: 
CS-DA02-25005
# Topic: 
Netflix dataset - discovery, cleaning, transformation, validation, export

Step 1: Import libraries and load dataset

In [1]:
import pandas as pd
import os

In [2]:
print(os.getcwd())   # Check current working directory

/kaggle/working


In [3]:
filepath = '/kaggle/input/netflix-shows/netflix_titles.csv'
df = pd.read_csv(filepath)    # Load Netflix dataset

In [4]:
print("Shape of dataset:", df.shape)
df.head()        # Show shape and first few rows

Shape of dataset: (8807, 12)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


Step 2: Data Discovery

In [5]:
df.info()  # Overview of the dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [6]:
print("Shape of dataset (rows, cols):", df.shape)     # Number of rows and columns

Shape of dataset (rows, cols): (8807, 12)


In [7]:
print("Columns:", df.columns.tolist())  # Column names

Columns: ['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration', 'listed_in', 'description']


In [8]:
print("Data types:\n", df.dtypes)  # Data types

Data types:
 show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object


In [9]:
print("Missing values per column:\n", df.isnull().sum())   # Missing values

Missing values per column:
 show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64


In [10]:
# % of missing values across entire dataset
print("Average missing %:", df.isnull().sum().mean() * 100)

# % of missing values per column
print("Missing % per column:\n", df.isnull().mean() * 100)

Average missing %: 35891.66666666667
Missing % per column:
 show_id          0.000000
type             0.000000
title            0.000000
director        29.908028
cast             9.367549
country          9.435676
date_added       0.113546
release_year     0.000000
rating           0.045418
duration         0.034064
listed_in        0.000000
description      0.000000
dtype: float64


In [11]:
print("Duplicate rows:", df.duplicated().sum())  # Duplicates

Duplicate rows: 0


Step 3: Structuring

In [12]:
# Make all column names lowercase and replace spaces with underscores
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [13]:
df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')  # Convert 'date_added' to datetime

In [14]:
# Separate duration into value and unit
df[['duration_value', 'duration_unit']] = df['duration'].str.extract(r'(\d+)\s*(\w+)')  

In [15]:
# Convert duration_value to numeric
df['duration_value'] = pd.to_numeric(df['duration_value'], errors='coerce')

In [16]:
# Preview changes
df[['duration', 'duration_value', 'duration_unit']].head()

,duration,duration_value,duration_unit
0,90 min,90.0,min
1,2 Seasons,2.0,Seasons
2,1 Season,1.0,Season
3,1 Season,1.0,Season
4,2 Seasons,2.0,Seasons


Step 4: Cleaning

In [17]:
# Remove duplicates
print("Duplicates before:", df.duplicated().sum())
df = df.drop_duplicates()
print("Duplicates after:", df.duplicated().sum())

Duplicates before: 0
Duplicates after: 0


In [18]:
# Drop unnecessary column
df = df.drop(columns=['description'])

In [19]:
# Fill missing values
df['director'] = df['director'].fillna('Not Given')
df['cast'] = df['cast'].fillna('Not Given')
df['country'] = df['country'].fillna('Not Given')

In [20]:
# Drop rows where date_added, rating, or duration are missing
df = df.dropna(subset=['date_added', 'rating', 'duration'])

Step 5: Error Checks

In [21]:
# Check if date_added year < release_year
invalid = df[df['date_added'].dt.year < df['release_year']]
print("Number of invalid records:", invalid.shape[0])

Number of invalid records: 14


In [22]:
# Display sample of invalid records
invalid[['title', 'release_year', 'date_added']].head()

,title,release_year,date_added
1551,Hilda,2021,2020-12-14
1696,Polly Pocket,2021,2020-11-15
2920,Love Is Blind,2021,2020-02-13
3168,Fuller House,2020,2019-12-06
3287,Maradona in Mexico,2020,2019-11-13


Step 6: Validation

In [23]:
# Drop helper columns if any
if 'dir_cast' in df.columns:
    df = df.drop(columns=['dir_cast'])

In [24]:
# Confirm datatypes
print(df.dtypes)

show_id                   object
type                      object
title                     object
director                  object
cast                      object
country                   object
date_added        datetime64[ns]
release_year               int64
rating                    object
duration                  object
listed_in                 object
duration_value           float64
duration_unit             object
dtype: object


In [25]:
# Sample rows
df.sample(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,duration_value,duration_unit
6422,s6423,Movie,Carol,Todd Haynes,"Cate Blanchett, Rooney Mara, Sarah Paulson, Ky...","United Kingdom, United States, Australia",2020-10-20,2015,R,119 min,"Dramas, LGBTQ Movies, Romantic Movies",119.0,min
8187,s8188,Movie,The American President,Rob Reiner,"Michael Douglas, Annette Bening, Martin Sheen,...",United States,2021-01-01,1995,PG-13,113 min,"Comedies, Dramas, Romantic Movies",113.0,min
3841,s3842,Movie,Wine Country,Amy Poehler,"Amy Poehler, Maya Rudolph, Ana Gasteyer, Rache...",United States,2019-05-10,2019,R,104 min,Comedies,104.0,min
2959,s2960,TV Show,Itaewon Class,Not Given,"Park Seo-joon, Kim Da-mi, Yu Jae-myeong, Kwon ...",South Korea,2020-02-01,2020,TV-MA,1 Season,"International TV Shows, Korean TV Shows, TV Dr...",1.0,Season
1746,s1747,TV Show,Can You Hear Me?,Not Given,"Mélissa Bédard, Ève Landry, Florence Longpré",Canada,2020-11-02,2020,TV-MA,2 Seasons,"International TV Shows, TV Comedies, TV Dramas",2.0,Seasons


Step 7: Export Final Clean Dataset

In [26]:
# Reset index
df = df.reset_index(drop=True)

In [27]:
# Export cleaned dataset
output_path = '/kaggle/working/cleaned_netflix.csv'
df.to_csv(output_path, index=False)
print("Cleaned dataset saved to:", output_path)

Cleaned dataset saved to: /kaggle/working/cleaned_netflix.csv
